# API project

## 1. Import libraries

In [1]:
import requests
from bson import ObjectId

## 1. Create Users

The objective of this request is to add a new user in the table `Users`. You can add few users with the same name, but they will have different **_id**.

In [2]:
users=["Geralt", "Triss", "Jaskier", "Filavandrel", "Toruviel", "Torque"]
for name in users:
    print(requests.get(f"http://localhost:3500/user/create/{name}").json())  

{'Result': 'Inserted Geralt - ID: 5eaf2e0d60759af624578a67'}
{'Result': 'Inserted Triss - ID: 5eaf2e0e60759af624578a68'}
{'Result': 'Inserted Jaskier - ID: 5eaf2e0e60759af624578a69'}
{'Result': 'Inserted Filavandrel - ID: 5eaf2e0f60759af624578a6a'}
{'Result': 'Inserted Toruviel - ID: 5eaf2e0f60759af624578a6b'}
{'Result': 'Inserted Torque - ID: 5eaf2e0f60759af624578a6c'}


In [3]:
users=["Tissaia", "Yennefer", "Fringilla", "Anica"]
for name in users:
    print(requests.get(f"http://localhost:3500/user/create/{name}").json())  

{'Result': 'Inserted Tissaia - ID: 5eaf2e0f60759af624578a6d'}
{'Result': 'Inserted Yennefer - ID: 5eaf2e0f60759af624578a6e'}
{'Result': 'Inserted Fringilla - ID: 5eaf2e0f60759af624578a6f'}
{'Result': 'Inserted Anica - ID: 5eaf2e0f60759af624578a70'}


## 2. Create Chat

Knowing the users (IDs) to be in the chat, this request introduces a row in the `Chats` table with the specified ids in the column **members**.

In [4]:
users_chat =['5eadcc0b4645f34974e56780',
             '5eadcc0b4645f34974e56781',
             '5eadcc0b4645f34974e56782',
             '5eadcc0b4645f34974e56783']
users_chat = ','.join(users_chat)
parameters = {"users_ids":users_chat}
requests.get("http://localhost:3500/chat/create", params=parameters).json()

{'result': 'You have created a chat',
 'Chat ID': '5eaf2e1060759af624578a71',
 'Members': 4}

What happens if you try to create a chat with users that are not in the database?

In [5]:
users_chat =['5ead8f1f2eb08989c88ecdc1',
             '5ead8f1f2eb08989c88ecdd2',
             '5ead8f1f2eb08989c88ecdd3']
users_chat = ','.join(users_chat)
parameters = {"users_ids":users_chat}
requests.get("http://localhost:3500/chat/create", params=parameters).json()

{'Result': 'You are trying to create a chat with users that are not in de DB'}

## 3. Add user in a pre-existing chat

Let's say a new member entered in one of the chats. Let's introduce this member in the members colum in the `Chats` table.

In [6]:
parameters = {"user_id":'5eaf2dc381e52ac0a17c3171'}
chat_id='5eadb94fbbbaa3faa1599730'
requests.get(f"http://localhost:3500/chat/{chat_id}/adduser", params=parameters).json()

{'Result': 'You have introduced a user',
 'In Chat': '5eadb94fbbbaa3faa1599730',
 'User ID': '5eaf2dc381e52ac0a17c3171'}

What happens if you try to add an user that is not in the database or is already a chat member?

In [7]:
parameters = {"user_id":'5ead8f1f2eb08989c88ecdd7'} #Not in the database
chat_id='5eadb94fbbbaa3faa1599730' 
requests.get(f"http://localhost:3500/chat/{chat_id}/adduser", params=parameters).json()

{'message': 'User_id is not in the DB', 'status': 'error'}

In [8]:
parameters = {"user_id":'5eada20b9b27ff529f8d56b7'} #Already chat member
chat_id='5eadb94fbbbaa3faa1599730' 
requests.get(f"http://localhost:3500/chat/{chat_id}/adduser", params=parameters).json()

{'message': 'User_id is already in the chat', 'status': 'error'}

What happens if you make a mistake and you put a chat id that is not in the database?

In [9]:
parameters = {"user_id":'5ead8f1f2eb08989c88ecdcf'} 
chat_id='5eada2509b27ff529f8d56ba' #Not in database
requests.get(f"http://localhost:3500/chat/{chat_id}/adduser", params=parameters).json()

{'message': 'Chat_id not found', 'status': 'error'}

## 4. Add message

With this request a row will be introduced in the table `Messages`. When introducing a message, a pre-trained neural network with NLTK sentiment analysis studies the content in the text and returns a score (positive, neutral or negative content).

In [10]:
#Yennefer
data={'user_id':'5eadcc0b4645f34974e56781',
      'text':'Like us?'}
chat_id='5eadcc454645f34974e56784'
requests.post(f"http://localhost:3500/chat/{chat_id}/addmessage", data=data).json()

{'Result': 'You have inserted a message',
 'With ID': '5eaf2e1260759af624578a72',
 'In Chat': '5eadcc454645f34974e56784',
 'From User': '5eadcc0b4645f34974e56781',
 'Name User': 'Yennefer',
 'Content': 'Like us?',
 'Date': '2020-05-03 22:48:18.563587'}

## 5. Obtain all the messages in a chat 

In [11]:
chat_id='5eadb94fbbbaa3faa1599730'
requests.get(f"http://localhost:3500/chat/{chat_id}/list").json()

[{'name': 'Jaskier', 'message': 'This is the part where we escape.'},
 {'name': 'Geralt', 'message': 'This is the part where they kill us.'},
 {'name': 'Jaskier', 'message': 'Who’s they?'},
 {'name': 'Toruviel', 'message': 'Beast!'},
 {'name': 'Geralt', 'message': 'Elves.'},
 {'name': 'Jaskier',
  'message': 'Oi, that’s my lute. Give that back. Quick, Geralt. Do your- your witchering-'},
 {'name': 'Geralt', 'message': ' Shut up!'},
 {'name': 'Jaskier', 'message': 'No.'},
 {'name': 'Toruviel', 'message': 'You shut up!'},
 {'name': 'Jaskier',
  'message': ' my Elder speech is rough. I only got part of that.'},
 {'name': 'Toruviel', 'message': 'Humans, shut up.'},
 {'name': 'Jaskier', 'message': 'Ah, got it, thanks so much.'},
 {'name': 'Toruviel', 'message': 'Do you wanna die right now?'},
 {'name': 'Geralt', 'message': 'As opposed to later?'},
 {'name': 'Jaskier', 'message': 'No, please, not the...'},
 {'name': 'Geralt', 'message': 'Leave off! He’s just a bard.'},
 {'name': 'Toruviel',


## 6. Sentiments in a chat

This request shows all the messages in a chat and the positive/neutral/negative score applied to them according to the NLTK lexicum applied (Vader in this case). It also shows the average scores for the whole chat. 

In [12]:
chat_id='5eadb94fbbbaa3faa1599730'
requests.get(f"http://localhost:3500/chat/{chat_id}/sentiment").json()

{'Average Score Chat': {'Negative': 0.26871875,
  'Neutral': 0.6617812500000002,
  'Positive': 0.0695},
 'Messages': [{'name': 'Jaskier',
   'message': 'This is the part where we escape.',
   'negative_score': 0.0,
   'neutral_score': 0.541,
   'positive_score': 0.459},
  {'name': 'Geralt',
   'message': 'This is the part where they kill us.',
   'negative_score': 0.61,
   'neutral_score': 0.39,
   'positive_score': 0.0},
  {'name': 'Jaskier',
   'message': 'Who’s they?',
   'negative_score': 0.0,
   'neutral_score': 1.0,
   'positive_score': 0.0},
  {'name': 'Toruviel',
   'message': 'Beast!',
   'negative_score': 0.0,
   'neutral_score': 1.0,
   'positive_score': 0.0},
  {'name': 'Geralt',
   'message': 'Elves.',
   'negative_score': 0.0,
   'neutral_score': 1.0,
   'positive_score': 0.0},
  {'name': 'Jaskier',
   'message': 'Oi, that’s my lute. Give that back. Quick, Geralt. Do your- your witchering-',
   'negative_score': 0.0,
   'neutral_score': 1.0,
   'positive_score': 0.0},
  {

What happens if you try to get the sentimens of a chat that is not in the DB?

In [13]:
chat_id='5eac4b041d3b47e21798aba4'
requests.get(f"http://localhost:3500/chat/{chat_id}/sentiment").json()

{'message': 'chat_id not found', 'status': 'error'}

## 7. Recommendations

The recommendation system applied uses the cosine similarity distance. It is thus a judgment of orientation and not magnitude: two vectors with the same orientation have a cosine similarity of 1, two vectors oriented at 90° relative to each other have a similarity of 0, and two vectors diametrically opposed have a similarity of -1, independent of their magnitude. All the words a user says are characterised by a vector where the value in each dimension corresponds to the number of times the term appears in the document.

In [14]:
user_id='5eada20b9b27ff529f8d56b5'
requests.get(f"http://localhost:3500/user/{user_id}/recommend").json()

{'Result': [['5eada20c9b27ff529f8d56b9', 0.37665889234460664],
  ['5eada20c9b27ff529f8d56ba', 0.29834709485930816],
  ['5eadcc0b4645f34974e56780', 0.28847487433165775]]}

What happens if the user id is not in the database?

In [15]:
user_id='5ead5c130cf284fc1a25cab0'
requests.get(f"http://localhost:3500/user/{user_id}/recommend").json()

{'message': 'user_id not found', 'status': 'error'}